In [ ]:
import numpy as np

from data_utils import compute_euclidean_distance, load_numpy_data, save_numpy_array

from constants import FC_PROCESSED_TRAIN_COMPLEX_FIELDS_PREFIX, \
                      FC_CROPPED_TRAIN_COMPLEX_FIELDS_PREFIX, \
                      FC_PREDICTED_TRAIN_COMPLEX_FIELDS_PREFIX, \
                      FC_PREDICTED_CROPPED_TRAIN_COMPLEX_FIELDS_PREFIX, \
                      FC_PROCESSED_TRAIN_OUTPUT_FLUXES_PREFIX, \
                      PSF_TRAIN_FILE_SUFFIXES, \
                      NUMPY_SUFFIX

# 1. Brute Force

In [2]:
def create_path(prefix):
    data_path = f"{prefix}{suffix}{NUMPY_SUFFIX}"
    return data_path

In [ ]:
for suffix in PSF_TRAIN_FILE_SUFFIXES:
    fluxes_path = create_path(FC_PROCESSED_TRAIN_OUTPUT_FLUXES_PREFIX)
    og_complex_fields_path = create_path(FC_PROCESSED_TRAIN_COMPLEX_FIELDS_PREFIX)
    og_cropped_complex_fields_path = create_path(FC_CROPPED_TRAIN_COMPLEX_FIELDS_PREFIX)
    predicted_complex_fields_path = create_path(FC_PREDICTED_TRAIN_COMPLEX_FIELDS_PREFIX)
    predicted_cropped_complex_fields_path = create_path(FC_PREDICTED_CROPPED_TRAIN_COMPLEX_FIELDS_PREFIX)
    
    fluxes = load_numpy_data(f"{}{suffix}{NUMPY_SUFFIX}")